## Data_collection

In [2]:
data = 'jeju_island'
print(data)

jeju_island


In [3]:
from requests import request
from requests.compat import urljoin, urlparse
from requests.exceptions import HTTPError
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
from time import sleep
import re

In [4]:
def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    method = method.upper() #method가 GET으로 받아야 하는데 get으로 받을 경우 방지
    if canfetch(url) == False:
        print('[Error] ' + url)
#     else: # 실제 수집할 때, 제약사항이 많으므로 여기선 잠시 해제
    try:
        resp = request(method, url,
               params=params if method=='GET' else {},
               data=params if method=='POST' else {},
               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(1) # => random
            resp = download(url, params, headers, method, limit-1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
    return resp

In [78]:
url = 'http://bus.jeju.go.kr/mobile/schedule/listSchedule'
resp = download(url)
dom = BeautifulSoup(resp.text, 'html.parser')

In [105]:
tmp_file = open('tmp_data.txt','w')

for _ in dom.find('ul', class_ = 'contents_body_wrap').find_all('li'):
    tmp_file.write(urljoin(url, _.find('a')['href']) + '\n')
    #print(_.find("span", class_ = "span_one").text)
    #print(urljoin(url, _.find("a")["href"]))
tmp_file.close()